In [ ]:
import pandas as pd

train = pd.read_csv('../../data/train.csv')
test = pd.read_csv('../../data/test.csv')

train.head()

In [ ]:
train_X = train.drop(columns=['ID','y'])
train_y = train['y']

train_data = pd.concat([train_X, train_y], axis=1)

In [ ]:
# normalize
from sklearn.preprocessing import MinMaxScaler

minmax = MinMaxScaler()

train_X_nor = minmax.fit_transform(train_X)

train_X_nor = pd.DataFrame(train_X_nor, columns=train_X.columns)

train_data_nor = pd.concat([train_X_nor, train_y], axis=1) # 인덱스 불일치로 인한 결합 오류 방지


In [ ]:
import sweetviz as sv

report = sv.analyze(train_X_nor)
report.show_html('report.html')


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns

# create histograms for each variables
fig,axes = plt.subplots(4,3,figsize=(20,20))
axes = axes.ravel() #  Converts a two-dimensional array to a one-dimensional array

for i, col in enumerate(train):
    sns.histplot(train[col], ax= axes[i], kde=True)
    axes[i].set_title(f'Distribution of {col}')

plt.tight_layout()
plt.show()
  
# create box-plots for each variable
fig,axes = plt.subplots(4,3,figsize=(20,20))
axes = axes.ravel()

for i, col in enumerate(train):
    sns.boxplot(y=train[col], ax= axes[i])
    axes[i].set_title(f'Box-plot of {col}')
    
plt.tight_layout()
plt.show()

In [ ]:
from pycaret.regression import setup, compare_models, pull, save_model, load_model, predict_model

regressor = setup(data = train_data_nor, target='y', session_id=42)
best_model = compare_models()
results = pull()

results

In [ ]:
save_model(best_model, 'best_model')

In [ ]:
test_X = test.drop(columns=['ID'])
best_model = load_model('best_model')
preds = predict_model(best_model, data = test_X)

preds

In [ ]:
submission = pd.read_csv('../../data/sample_submission.csv')
submission['y'] = preds['prediction_label']
submission.to_csv('../normalized_AutoML/nor_AutoML_submission.csv',index=False)